### Configurando un entorno virtual e Importar las bibliotecas necesarias


In [1]:
# Configuracio del entorno virtual

!pip install virtualenv
!virtualenv my_env # crea un entorno virtual llamado my_env

# El comando 'source' se usa para activar un entorno virtual en una terminal.
# En Google Colab, activar un entorno virtual de esta manera no se mantiene
# entre diferentes ejecuciones de celdas debido a la naturaleza sin estado
# del entorno.
# Para instalar paquetes, normalmente es más directo usar !pip install.
# Si se necesita un intérprete de Python específico del entorno virtual,
# generalmente se lo invoca explícitamente (por ejemplo:
# !my_env/bin/python -m pip install).
# Por ahora, comentaremos el comando 'source' ya que su efecto puede
# malinterpretarse en el contexto de Colab.
# !source my_env/bin/activate  # activar my_env


created virtual environment CPython3.12.12.final.0-64 in 905ms
  creator CPython3Posix(dest=/content/my_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.3
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [2]:
# Instalacion de librerias necesarias

# installing necessary packages in my_env
!pip install \
gradio==4.44.0 \
ibm-watsonx-ai==1.1.2  \
langchain==0.2.11 \
langchain-community==0.2.10 \
langchain-ibm==0.1.11 \
chromadb==0.4.24 \
pypdf==4.3.1 \
pydantic==2.9.1 \
huggingface_hub==0.23.0

In [3]:
# Impoprtacion de librerias Watsonx
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from ibm_watsonx_ai import Credentials
from langchain_ibm import WatsonxLLM, WatsonxEmbeddings

In [4]:
# Importacion de liberias LangChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA

In [5]:
#Imoptacionde HuggingFace y Gradio
from huggingface_hub import HfFolder
import gradio as gr

In [6]:
#Suprimir Warnings

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

## Inicializacion del LLM

 Se inicializa el LLM creando una instancia, en este caso, WatsonxLM, una clase de langchain_ibm
 Watson puede usar distintos modelos fundamentales subyasentes (Son los cimientos matemáticos y arquitectónicos que permiten que un LLM exista y aprenda.), en este caso Mixtral 8x7B.
 Tambien se podrian usar otros modelos como Llama3.3 70B.

Para inicializar el LLM, utiliza el siguiente código en qabot.py.
Tene en cuenta que se requiere inicializar el modelo con una temperatura de 0.5 y establecer el límite máximo de generación de tokens en 256.


In [19]:
## LLM IBMWatsonx ##



def get_llm():

  # CONFIGURACION DE PARAMETROS DEL MODELO (como se controla el modelo)
  parametros = {
    #Decodificacion "greedy" elige la palabra mas probable (codiciosa)
    #Mejor para respuestas serias. ('sample' es mejor para creatividad.)
    GenParams.DECODING_METHOD: "greedy",

    #Cantidad de texto que puede generar como maximo
    GenParams.MAX_NEW_TOKENS: 200,

    #Temperatura: número (generalmente entre 0 y 2) que ajusta cuánta aleatoriedad usa el modelo al elegir la siguiente palabra

    # + temp mas baja (0 - 0.3) mas presiso, logico y reptible
    # + temp media (0.5 - 0.8) es un equilibrio entre coherencia y creatividad
    # + temp alta (1.0 - 2.0) mayor creatividad, impredisible e incluso caotico

    GenParams.TEMPERATURE: 0.1, ## Para un bot de QA sobre documentos, queremos precisión (cerca de 0).

    #Penalizacion por repaticion de palabras
    GenParams.REPETITION_PENALTY: 1.1
    }

  # Definicion del modelo (ID)
  modelo_id = 'ibm/granite-3-2-8b-instruct'

  model_id = modelo_id
  parameters = parametros
  project_id = "skills-network"
  watsonx_llm = WatsonxLLM(
      model_id=model_id,
      url="https://us-south.ml.cloud.ibm.com",
      project_id=project_id,
      params=parameters,
  )
  return watsonx_llm

print("LLM Watsonx inicializado correctamente.")

LLM Watsonx inicializado correctamente.


### Tarea 1: Carga de Documentos (Document Loader)



In [20]:
# Document Loader
def document_loader(file):

  #Inicializar el cargador con nombre o ruta del archivo
  #file.name otiene la ruta temporal donde gradio guardo el PDF que subio
  loader = PyPDFLoader(file.name)

  #Carga de contenido, devulve lista de objetos Document (uno por pagina)
  loaded_document = loader.load()

  return loaded_document

### Tarea 2: Aplicar Tecnicas de division de texto (Definir el divisor de texto)

In [21]:
#El cargador carga el contenido, pero su metodo .load(), NO LO DIVIDE EN FRACMENTOS
#Se define un divisor de documentos

#Text Splitter
def text_splitter(data):
  """
  Recibe una lista de documentos cargados (data) y devuelve una lista de fragmentos (chunks).
  """
  #Es el más inteligente: intenta cortar por párrafos, luego frases, luego palabras, para no romper ideas.
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size = 1000,
      chunck_overlap = 200,
      length_function = len
  )

  # Ejecucion del corte
  # .split_documents toma la lista de documentos (con metadatos incluidos)
  chunks = text_splitter.split_documents(data)

  return chunks

### Tarea 3: Incrustar Documentos (Definir el modelo de Embedding)

In [22]:
#Este modelo convierte fragmentes de textos en representaciones vectoriales
def embedding_generator_watsonx():
    """
    Inicializa y devuelve el modelo de embeddings de Watsonx (Slate).
    """
    #Configuracionde parametros para embeddings
    embed_params = {
        EmbedTextParamsMetaNames.TRUNCATE_INPUT_TOKENS: 3, #Limita el texto de entrada a 'x' tokens.
        EmbedTextParamsMetaNames.RETURN_OPTIONS: {"input_text": True} #Hace que la API devuelva el texto final usado (útil para ver truncamientos).
    }

    #Creacion del modelo

    embedding_model = WatsonxEmbeddings(
        model_id="ibm/slate-125m-english-rtrvr-v2",
        url="https://us-south.ml.cloud.ibm.com",
        project_id="skills-network",
        params=embed_params
    )

    return embedding_model


### Tarea 4: Crear y configurar bases de datos vectoriales para almacenar embeddings (Definicion del Almacen Vectorial)


In [29]:
def vector_database(chunks):
  """
  Toma los fragmentos de texto y el modelo de embeddings, y crea la base de datos vectorial.
  """


  embedding_model = embedding_generator_watsonx()
  vectordb = Chroma.from_documents(
      documents=chunks, # Los trozos de texto
      embedding=embedding_model # El modelo de embedding convertidor
  )
  return vectordb

### Tarea 5: Desarrollar un recuperador para obtener segmentos de documentos basados en consultas (Definir el recuperador)

In [31]:
# La funcion retriever conecta la CARGA, DIVISION Y BASE DE DATOS, en una sola PIPLINE automatica.

##Retriever
def retriever(file):
  # Carga de archivo (T1)
  split = document_loader(file)

  # Division del texto (T2)
  # pasamos split como entrada
  chunks = text_splitter(split)

  # Creacion de la base de datos e embeddings de los vectores (T4)
  # pasamos chunks como entrada
  vectordb = vector_database(chunks)

  # Se convierte la base de datos en un buscador de archivos
  retiever = vectordb.as_retriever()

  return retriever

###Tarea 6: Construir un bot de preguntas y respuestas que aproveche LangChain y LLM para responder preguntas

#### Definir una cadena de preguntas y respuestas

In [32]:
## QA Chain
def retriever_qa(file, query):
  print("Starting retriever_qa function...")
  #Llamamos al cerebro LLM
  llm = get_llm()
  print("LLM initialized.")

  #Llamamos al Retriever (Buscador)
  retriever_obj = retriever(file) #(T5)
  print("Retriever object created.")

  #Creacion de la cadena RAG (RetrieverQA)
  qa = RetrievalQA.from_chain_type(
      llm=llm,
      chain_type="stuff", #"stuff" mete el texto en el prompt
      retriever=retriever_obj, #Usamos el objeto retriever creado arriba
      return_source_documents=False #False para que solo devuelva texto (limpio)
  )
  print("RetrievalQA chain created.")

  #Ejecucion de la pregunta, se pasa query (pregunta) al modelo invoke
  response = qa.invoke(query)
  print(f"Response from QA chain: {response}")

  #Resultado del texto
  return response["result"]

### Tarea 7: Interfaz y Lanzamiento (Gradio)

In [37]:
rag_application = gr.Interface(
    fn=retriever_qa,
    allow_flagging="never",
    inputs=[
        gr.File(label="Upload PDF File", file_count="single", file_types=['.pdf'], type="filepath"),
        gr.Textbox(label="Input Query", lines=2, placeholder="Type your question here...")
    ],
    outputs=gr.Textbox(label="Answer"),
    title="RAG Chatbot",
    description="Upload a PDF document and ask any question. The chatbot will try to answer using the provided document."
)

rag_application.launch(server_name="0.0.0.0", server_port=7862)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6ed9604ec89c8f80e6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
